In [2]:
!pip install torch
!pip install git+https://github.com/openai/shap-e.git
!pip install pytorch3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [4]:
from shap_e.util.notebooks import decode_latent_mesh

In [ ]:
print(torch.version.cuda)

12.4


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:12<00:00, 72.0MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [30]:
batch_size = 4
guidance_scale = 15.0
prompt = "chair"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [31]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAALS0s7S0srSzs7SzsrOzs7OzsrKzsrOysrOysbKysbKxsLGysb…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAEkiB0ghBkchBkchBUYhB0YhBkYhBUcgBkYgBkUhB0UhBkUhBU…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAFclE1UjFFUjE1giElUiE1QiEVMiE1IiE1IiElQhE1MhE1IhE1…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAIE8GoE7GoE7GYA8GoA7GX47GYA6GX86GX46GX46GH08Gn07Gn…

In [33]:
for i, latent in enumerate(latents):
  image = decode_latent_mesh(xm, latent)
  display(image)

TorchMesh(verts=tensor([[-0.4893,  0.3798,  0.9845],
        [-0.4931,  0.3953,  0.9845],
        [-0.4896,  0.4109,  0.9690],
        ...,
        [ 0.4884,  0.4419,  0.9771],
        [ 0.4884,  0.4574,  0.9615],
        [ 0.4884,  0.4574,  0.9703]], device='cuda:0'), faces=tensor([[12164,  5888,     0],
        [ 5888, 12165,     0],
        [    0,     1, 12164],
        ...,
        [ 5887, 20441, 20439],
        [20440, 12163,  5887],
        [ 5887, 12163, 20441]], device='cuda:0'), vertex_channels={'R': tensor([0.2848, 0.2761, 0.3153,  ..., 0.4157, 0.3155, 0.3793], device='cuda:0'), 'G': tensor([0.2903, 0.2799, 0.3060,  ..., 0.4170, 0.3190, 0.3749], device='cuda:0'), 'B': tensor([0.2836, 0.2761, 0.3050,  ..., 0.4143, 0.3252, 0.3756], device='cuda:0')}, face_channels={})

TorchMesh(verts=tensor([[-0.5201, -0.4419,  0.8760],
        [-0.5209, -0.4419,  0.8915],
        [-0.5212, -0.4419,  0.9070],
        ...,
        [ 0.5349,  0.4574, -0.0354],
        [ 0.5349,  0.4729, -0.0725],
        [ 0.5349,  0.4729, -0.0470]], device='cuda:0'), faces=tensor([[17382,  9890,     0],
        [ 9890,  9891,     0],
        [ 9891,     1,     0],
        ...,
        [17380, 17381,  9888],
        [17381,  9889,  9888],
        [ 9889, 17381, 27159]], device='cuda:0'), vertex_channels={'R': tensor([0.0445, 0.0452, 0.0448,  ..., 0.0459, 0.0537, 0.0523], device='cuda:0'), 'G': tensor([0.0058, 0.0059, 0.0060,  ..., 0.0054, 0.0058, 0.0057], device='cuda:0'), 'B': tensor([0.0012, 0.0012, 0.0013,  ..., 0.0010, 0.0011, 0.0011], device='cuda:0')}, face_channels={})

TorchMesh(verts=tensor([[-0.6745, -0.4419,  0.9535],
        [-0.6744, -0.4419,  0.9690],
        [-0.6768, -0.4264,  0.9535],
        ...,
        [ 0.6899,  0.4419, -0.7580],
        [ 0.6899,  0.4574, -0.8608],
        [ 0.6899,  0.4574, -0.7514]], device='cuda:0'), faces=tensor([[ 9404,  6654,     0],
        [ 6654,  6655,     0],
        [ 6655,     1,     0],
        ...,
        [ 9402,  9403,  6652],
        [ 9403,  6653,  6652],
        [ 6653,  9403, 18049]], device='cuda:0'), vertex_channels={'R': tensor([0.0470, 0.0478, 0.0510,  ..., 0.0651, 0.0701, 0.0617], device='cuda:0'), 'G': tensor([0.0044, 0.0045, 0.0045,  ..., 0.0058, 0.0060, 0.0053], device='cuda:0'), 'B': tensor([0.0024, 0.0024, 0.0024,  ..., 0.0030, 0.0030, 0.0028], device='cuda:0')}, face_channels={})

TorchMesh(verts=tensor([[-0.4731, -0.5039, -0.1318],
        [-0.4732, -0.4884, -0.1318],
        [-0.4731, -0.4884, -0.1163],
        ...,
        [ 0.4729,  0.4884,  0.8246],
        [ 0.4729,  0.5039,  0.7837],
        [ 0.4729,  0.5039,  0.8348]], device='cuda:0'), faces=tensor([[15500,  7328,     0],
        [ 7328, 15501,     0],
        [    0,     1, 15500],
        ...,
        [15498, 15499,  7326],
        [15499,  7327,  7326],
        [ 7327, 15499, 25267]], device='cuda:0'), vertex_channels={'R': tensor([0.1539, 0.1681, 0.1630,  ..., 0.1919, 0.1703, 0.1751], device='cuda:0'), 'G': tensor([0.0201, 0.0221, 0.0214,  ..., 0.0237, 0.0221, 0.0227], device='cuda:0'), 'B': tensor([0.0037, 0.0040, 0.0040,  ..., 0.0044, 0.0043, 0.0043], device='cuda:0')}, face_channels={})

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)

/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(
